<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, lon, launch in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    # Circle with popup showing the site name
    folium.Circle(location=[lat, lon],
                  radius=1000,
                  color='#d35400',
                  fill=True,
                  popup=launch
                  ).add_to(site_map)

    # Label-style marker (text)
    folium.map.Marker(
        location=[lat, lon],
        # Create an icon like a text label
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{launch}</b></div>',
        )
    ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


> **Answer**:<br>
> - **Are all launch sites in proximity to the Equator line?**
>   - **No**.
>   - KSC LC-39A and CCAFS SLC-40 (Florida) are at ~28.5°N.
>   - VAFB SLC-4E (California) is at ~34.7°N.
>   - These are **mid-latitudes**, not equatorial (0–10°). Florida does benefit somewhat from **Earth’s rotational speed** for **eastward** launches, but it’s not “near the Equator” like, for example, Kourou (~5°N). Vandenberg is also used a lot for polar/SSO orbits, where **that equatorial boost matters less**.
> - **Are all launch sites in very close proximity to the coast?**
>   - **Yes**.
>   - All three sites are **right by the sea** (two on Florida’s Atlantic coast and one on California’s Pacific coast). This is done for:
>     - **Safety**: trajectories and stage drop zones over the **ocean** (sparsely populated areas).
>     - **Clear launch corridors** (Florida toward the **Atlantic** for eastward launches; Vandenberg toward the **Pacific** for southbound polar launches).
>     - **Recovery**: use of drone ships and coastal landing zones.
>   - You can see it clearly on the map: the Florida markers sit on Cape Canaveral’s barrier islands, and VAFB is on the California coast.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f'{record['Launch Site']} - {'Success' if int(record['class']) == 1 else 'Failure'}',
        # Pin color based on the marker_color column ('green' / 'red')
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


> **Answer**:<br>
> From the map we can see:
> - **KSC LC-39A: high success rate**. The cluster shows **many more green markers** than red ones.
> - **CCAFS SLC-40: also high/moderately high**. Greens predominate, though there are some reds.
> - **VAFB SLC-4E: intermediate**. A more balanced mix of green and red.
> - **CCAFS LC-40: the lowest** of the four. The cluster shows quite a few reds.
> In short:
> - “Relatively high”: KSC LC-39A and, second, CCAFS SLC-40. The other two lag behind.

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
site_name = 'VAFB SLC-4E'

coast_lat = 34.63380
coast_long = -120.64700

distance_coastline = calculate_distance(34.632834, -120.610745, coast_lat, coast_long)
print(f'Distance from {site_name} to the coast: {distance_coastline:.2f} km')

Distance from VAFB SLC-4E to the coast: 3.32 km


In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
distance_marker = folium.Marker(
    location=[coast_lat, coast_long],
    icon=DivIcon(
        icon_size=(80, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>',
    )
)

distance_marker.add_to(site_map)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# # Coordinates: [ [lat_site, lon_site], [lat_costa, lon_costa] ]
coordinates = [
    [34.632834, -120.610745],
    [coast_lat, coast_long]
]

# # Create the line
lines = folium.PolyLine(
    locations=coordinates,
    color='blue',      # optional
    weight=1,          # line thickness
    opacity=0.7        # optional
)

site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [19]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# --- 1) Dictionary with target locations by launch site ---
targets_by_site = {
    'VAFB SLC-4E': [
        {'label': 'train', 'lat': 34.63663, 'lon': -120.62370},
        {'label': 'city', 'lat': 34.63773, 'lon': -120.46646},
        {'label': 'highway', 'lat': 34.62010, 'lon': -120.58020},
    ],
    'KSC LC-39A': [
        {'label': 'train', 'lat': 28.57035, 'lon': -80.80410},
        {'label': 'city', 'lat': 28.61200, 'lon': -80.80700},
        {'label': 'highway', 'lat': 28.58500, 'lon': -80.65000},
        {'label': 'coast', 'lat': 28.60080, 'lon': -80.58815},
    ],
    'CCAFS LC-40': [
        {'label': 'train', 'lat': 28.56253, 'lon': -80.58710},
        {'label': 'city', 'lat': 28.39220, 'lon': -80.66770},
        {'label': 'highway', 'lat': 28.56206, 'lon': -80.57059},
        {'label': 'coast', 'lat': 28.56205, 'lon': -80.56772},
    ],
    'CCAFS SLC-40': [
        {'label': 'train', 'lat': 28.56302, 'lon': -80.58697},
        {'label': 'city', 'lat': 28.39220, 'lon': -80.60770},
        {'label': 'highway', 'lat': 28.56340, 'lon': -80.57078},
        {'label': 'coast', 'lat': 28.56330, 'lon': -80.56796},
    ],
}

In [18]:
# --- 2) Helper function: adds distance annotations and draws lines for all sites ---
def add_distance_annotations(site_map, launch_sites_df, targets_by_site, line_color='blue'):
    for site_name, targets in targets_by_site.items():
        # site coordinates
        sel = launch_sites_df.loc[launch_sites_df['Launch Site'] == site_name]
        if sel.empty:
            print(f'Warning: {site_name} is not in launch_sites_df')
            continue
        launch_lat, launch_lon = float(sel.iloc[0]['Lat']), float(sel.iloc[0]['Long'])

        # (optional) small marker at the site
        folium.CircleMarker([launch_lat, launch_lon], radius=3, color=line_color,
                            fill=True, fill_color=line_color, fill_opacity=0.9).add_to(site_map)

        # iterate through the site's targets
        for t in targets:
            d_km = calculate_distance(launch_lat, launch_lon, t['lat'], t['lon'])

            # line: site → target
            folium.PolyLine([[launch_lat, launch_lon], [t['lat'], t['lon']]],
                            color=line_color, weight=1, opacity=0.7).add_to(site_map)

            # text marker showing the distance at the target
            folium.Marker(
                [t['lat'], t['lon']],
                icon=DivIcon(
                    icon_size=(160, 24), icon_anchor=(0, 0),
                    html=f'<div style="font-size:12px;color:#d35400;"><b>{d_km:.2f} KM</b></div>'
                ),
                popup=f'{t["label"]} ({t["lat"]:.5f}, {t["lon"]:.5f})'
            ).add_to(site_map)

In [20]:
# --- 3) Function call ---
add_distance_annotations(site_map, launch_sites_df, targets_by_site)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


> **Answer**:<br>
>
> - **Proximity to rail lines?**
>   - VAFB SLC-4E: Yes, ~1–2 km to the west.
>   - CCAFS (LC-40/SLC-40): Yes, ~1 km (internal spurs/old tracks).
>   - KSC LC-39A: Not nearby; the main rail line (FEC on the mainland) is far (~15–20 km).
> 
> Conclusion: Except for KSC, the others have nearby rail access.
>
> - **Proximity to highways/main roads?**
>   - CCAFS (LC-40/SLC-40): Yes, <1 km to SR-3 and internal roads.
>   - KSC LC-39A: Yes, ~2–3 km to SR-3 (Kennedy Pkwy).
>   - VAFB SLC-4E: Yes, ~3–5 km to CA-1 (Cabrillo Hwy).
>
> Conclusion: All four sites are near major roads, especially CCAFS.
>
> - **Proximity to the coastline?**
>   - CCAFS (LC-40/SLC-40): Yes, ≈1 km.
>   - KSC LC-39A: Yes, ~5–7 km.
>   - VAFB SLC-4E: Yes, ~2–4 km.
>
> Conclusion: All sites are coastal (for safety and launch corridors over the ocean).
>
> - **Do they keep a distance from cities?**
>   - VAFB SLC-4E → Lompoc: ~10–15 km.
>   - KSC/CCAFS → Cape Canaveral/Titusville: ~10–25 km (depending on the point).
>
> Conclusion: Yes, they maintain a clear buffer from the nearest urban centers.
>
> **Overall conclusions**
> 
> - All four sites are **designed to operate near the ocean**, with **good ground access**, while keeping a **safe distance from cities**.
> - **CCAFS** is the most “compact” (coast and highway very close), ideal for eastward launches (LEO/GTO) and recovery at LZ-1/ASDS.
> - **KSC LC-39A** remains coastal and well-connected, though the rail line isn't adjacent to the pad; it's a hub for complex missions (crewed, heavy payloads).
> - **VAFB SLC-4E** combines nearby coast and reasonably close roads, and is well isolated from large cities — ideal for polar/SSO launches over the Pacific.
>
> **Summary**: ***Close to the coast and major roads, a degree of separation from cities, and*** (except LC-39A) ***nearby rail access***. This setup ***minimizes risk and optimizes logistics and first-stage recovery***.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
